<a href="https://colab.research.google.com/github/YogeshBarude/Prediction-using-Supervised-ML/blob/main/Task-7_Stock_Market_Prediction_using_Numerical_and_Textual_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GRIP internship 

## Task7 : #7 STOCK MARKET PREDICTION USING NUMERICAL AND TEXTUAL ANALYSIS

### Task :
* Create a hybrid model for stock price/performance prediction using numerical analysis of historical stock prices, and   sentimental analysis of news headlines

# By Yogesh Barude

* In this I have predicted if a companies stock will increase or decrease based on news headlines using sentiment analysis.

* This model will determine if the price of a stock will increase or decrease based on the sentiment of top news article headlines for the current day using Python and machine learning.

* I have used both numerical and textual data for this.

 (i) Time series analysis is performed on the Stock data.
 
 (ii) Sentiment analysis is performed on the News data.
 
 (iii) An analysis is performed by merging both the data to predict if the Close price of the stock will increase or decrease.



   ## Stock Market Prediction

In [ ]:
!pip install pandas-datareader
!pip install pmdarima
! pip install textBlob
! pip install vaderSentiment

     |████████████████████████████████| 1.4 MB 5.3 MB/s 
     |████████████████████████████████| 9.8 MB 40.5 MB/s 
     |████████████████████████████████| 233 kB 67.6 MB/s 
  Attempting uninstall: patsy
    Found existing installation: patsy 0.5.1
    Uninstalling patsy-0.5.1:
      Successfully uninstalled patsy-0.5.1
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
     |████████████████████████████████| 125 kB 5.0 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
%matplotlib inline
import seaborn as sns
import datetime
import pandas_datareader.data as web
import matplotlib as mpl
from matplotlib import style
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from statsmodels.tsa.seasonal import seasonal_decompose


from textblob import TextBlob
from matplotlib.pyplot import figure
from matplotlib import rcParams
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
import xgboost 
import lightgbm

## Importing stocks data from web

In [ ]:
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime.today()

In [ ]:
stocks = web.DataReader("AAPL", 'yahoo', start, end)

RemoteDataError: ignored

In [ ]:
stocks.info()

In [ ]:
stocks.head()

In [ ]:
stocks.reset_index(inplace=True)

In [ ]:
stocks.tail()

In [ ]:
stocks.shape

In [ ]:
stocks.describe()

### Data cleaning

In [ ]:
stocks['Date'] = pd.to_datetime(stocks['Date'])
stocks.head()

### Missing Values

In [ ]:
stocks.isna().sum()

### Visualizing the data -Close Price

In [ ]:
plt.figure(figsize=(5,5))
plt.title('Closing Price of Stocks', fontsize = 18)
plt.xlabel('Days', fontsize= 18)
plt.ylabel('Close', fontsize = 18)
plt.plot(stocks['Close'])
plt.show()

### Visualizing the data -Open Price

In [ ]:
plt.figure(figsize=(5,5))
plt.grid(True)
plt.plot(stocks['Open'])
plt.xlabel('Days')
plt.ylabel('Open Price')
plt.title('Opening price of Stocks')
plt.show()

### Visualising Stocks returns

In [ ]:
close = stocks['Close']
returns = close / close.shift(1) - 1

plt.figure(figsize = (10,6))
returns.plot(label='Return', color = 'g')
plt.title("Stock Returns")

## Time series Analysis -For Close Price
We can also perform the same analysis for Open price as well

   Time series decomposition involves thinking of a series as a combination of level, trend, seasonality, and noise components.
 we need to separate seasonality and trend from our series. The resultant series will become stationary through this process.

### Splitting data into train and test data

In [ ]:
train = stocks[:1600]
test = stocks[1600:]

In [ ]:
train.shape, test.shape

### Decomposition of Time series

### Stationarity

A stationary process has a mean and variance that do not change overtime and the process does not have trend.

The above time series does not look stationary.

To confirm that we will use “Dickey-Fuller test” to determine stationarity.

Dickey-Fuller test for variable

### Dickey-Fuller test

In [ ]:
def adfullerTest(X):
    result = adfuller(X,autolag = 'AIC')
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('No of Lags Used: %f' % result[2])
    print('Number of Obs Used: %f' % result[3])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))
    if result[1] <=0.05 :
         print("Reject against the null hypothesis, time series is stationary")
    else:
        print("Accept null hypothesis, time series is non-stationary ") 

In [ ]:
adfullerTest(train['Close'])


We can see that our statistic value of -1.4 is greater than the value of -3.437 at 1% critical value.
   By Comparing the test statistic to the critical values, it looks like we would have to fail to reject the null hypothesis that the time series is non-stationary and does have time-dependent structure.

##### Rolling Statistics

In [ ]:
rolling_mean_50 = (train['Close']).rolling(window=50).mean()
rolling_std_50 = (train['Close']).rolling(window=50).std()
plt.figure(figsize = (8,8))
plt.plot((train['Close']), color = 'blue', label = 'original')
plt.plot(rolling_mean_50, color = 'red', label = 'rolling mean 50')
plt.plot(rolling_std_50, color = 'black', label = 'rolling std 50')
plt.xlabel('Time')
plt.legend()
plt.title('Mean and Standard Deviation on  transformed data',  fontsize = 20)


Since the data shows changing variance over time, the first thing we will do is stabilize the variance by applying log transformation using the log() function. The resulting series will be a linear time series.

### Log Transfromation

Let’s log transform the dataset again to make the distribution of values more linear and better meet the expectations of this statistical test.

In [ ]:
train_log = np.log(train['Close']) 
test_log = np.log(test['Close'])

In [ ]:
from numpy import log
adfullerTest(log(train['Close']))

Running this shows a negative value for the test statistic.

We can see that the value is larger than the critical values, again, meaning that we can fail to reject the null hypothesis and in turn that the time series is non-stationary.

In [ ]:
rolling_mean_50 = log(train['Close']).rolling(window=50).mean()
rolling_std_50 = log(train['Close']).rolling(window=50).std()
plt.figure(figsize = (8,8))
plt.plot(log(train['Close']), color = 'g', label = 'Log Transformed')
plt.plot(rolling_mean_50, color = 'r', label = 'rolling mean 50')
plt.plot(rolling_std_50, color = 'r', label = 'rolling std 50')
plt.xlabel('Time')
plt.legend()
plt.title('Mean and Standard Deviation on Log transformed data',  fontsize = 20)


##### To Removing Linear Trend

###### We will now perform the first difference transformation to our series to remove the linear trend.

In [ ]:
mean_log = log(train['Close']).rolling(50).mean()

In [ ]:
train_log_diff = log(train['Close']) - mean_log
train_log_diff.dropna(inplace = True)
adfullerTest(train_log_diff)

In [ ]:
data=   train_log_diff
mean = data.rolling(50).mean()
std = data.rolling(50).std()
plt.figure(figsize = (8,8))
plt.plot(data, color = 'g', label = 'Differential Log Transformed data')
plt.plot(mean, color = 'r', label = 'rolling mean')
plt.plot(std, color = 'b', label = 'rolling std')
plt.xlabel('Time')
plt.legend()
plt.title('Mean and Standard Deviation on Differential Log Transformed data',  fontsize = 20)

Rejecting the null hypothesis means that the process has no unit root, and in turn that the time series is stationary or does not have time-dependent structure.
Here we can also observe that there is no such trend in mean and Standard deviation
So the now time series is statinary

Now the data is stationary we can apply ARIMA model to our data

### ARIMA Model 

    AR: Autoregression. A model that uses the dependent relationship between an observation and some number of lagged observations.
    I: Integrated. The use of differencing of raw observations (e.g. subtracting an observation from an observation at the previous time step) in order to make the time series stationary.
    MA: Moving Average. A model that uses the dependency between an observation and a residual error from a moving average model applied to lagged observations.

The parameters of the ARIMA model are defined as follows:

    p: The number of lag observations included in the model, also called the lag order.
    d: The number of times that the raw observations are differenced, also called the degree of differencing.
    q: The size of the moving average window, also called the order of moving average.

In [ ]:
import pmdarima as pmd

def arimamodel(timeseriesarray):
    autoarima_model = pmd.auto_arima(timeseriesarray, 
                             # start_p=1, 
                              #start_q=1,
                              #test="adf",
                              trace=True,
                              error_action = 'ignore',
                              suppress_warnings = True)
    return autoarima_model

In [ ]:
stocks_arima = arimamodel((train_log))
stocks_arima.summary()

In [ ]:
stocks_arima.plot_diagnostics(figsize=(10,10))
plt.show()

### Predictions on test data

In [ ]:
predict_ClosePrice = stocks_arima.predict(n_periods = len(test_log))
predict_ClosePrice = pd.DataFrame(predict_ClosePrice,index = test_log.index,columns=['predict_ClosePrice'])

In [ ]:
plt.plot(train_log, label='Train')
plt.plot(test_log, label='Test')
plt.plot(predict_ClosePrice, label='Prediction')
plt.title(' Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Actual Stock Price')

### Validation of the Arima Model

In [ ]:
print('Mean Squared Error      ',mean_squared_error(test_log, predict_ClosePrice))
print('Root Mean_Squared_Error ',np.sqrt(mean_squared_error(test_log, predict_ClosePrice)))
print('Mean Absolute Error     ',mean_absolute_error(test_log, predict_ClosePrice))
print('R-Squared               ',r2_score(test_log, predict_ClosePrice))

## Analyzing  news dataset 

In [ ]:
news=pd.read_csv("C:\\Users\\Keerthi\\Desktop\\04-Keerthi\\00-Spark Foundation\\Stocks\\india-news-headlines.csv")

In [ ]:
news.head()

In [ ]:
news['publish_date'] = pd.to_datetime(news['publish_date'],format= '%Y%m%d')
news.head()

In [ ]:
news.isna().sum()

In [ ]:
news.shape

In [ ]:
(news.columns)

### Analaysing "HEADLINE_CATEGORY" with "CITIES"

In [ ]:
news['headline_category'].value_counts().head()

In [ ]:
cities = news[news['headline_category'].str.contains('^city\.[a-z]+$', regex=True)]
cities.head(10)

In [ ]:
city = pd.DataFrame(columns = ['city_name'])
city['city_name'] = cities.headline_category.str.split('.',expand = True)[1]
cities = pd.concat([cities, city], axis = 1)
cities.head()


In [ ]:
cities.drop('headline_category', inplace =True,axis =1)


In [ ]:
cities.head()

In [ ]:
cites = cities.groupby(cities['city_name']).agg({'headline_text':'count'})
cites.head()

In [ ]:
cites.rename(columns = {'headline_text':'headline_count'}, inplace = True)

In [ ]:
cites = cites.sort_values(by='headline_count',ascending=False)
cites.head()

In [ ]:
top10cites = cites.head(10)


In [ ]:
def fig_plot(top10cites,title1):
    fig = px.line(top10cites,title =title1)
    for i in top10cites.columns[0:]:
        fig.add_bar(x= top10cites.index ,y = top10cites['headline_count'],name = i)
    fig.show()


In [ ]:
fig_plot(top10cites,'Count of Headlines for top10 Cities')

In [ ]:
cities.head()


### Analaysing "HEADLINE_CATEGORY" with "CATEGORIES"

In [ ]:
news.head()

In [ ]:
news['category']=news['headline_category'].str.split('.').map(lambda x : x[0])

In [ ]:
categories = news.groupby(['category']).agg({'headline_text':'count'}).sort_values(by='headline_text',ascending = False)
news_cat=categories.head(10)
news_cat.reset_index(inplace = True)
news_cat

In [ ]:
import matplotlib.colors as mcolors
plt.figure(figsize=(17,5))
plt.bar(news_cat.category,height= news_cat.headline_text, color = 'c')
plt.xlabel('Category')
plt.ylabel('Number of articles')
plt.title('Top 10 Categories')
plt.show()

In [ ]:
news.drop('headline_category', inplace  = True, axis =1)
news.head()

### Cleaning the data -Removing Stop words

In [ ]:
headline_text = ' '.join(news['headline_text'].str.lower())

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
wordcloud = WordCloud(stopwords=stop_words, background_color="white", max_words=1000).generate(headline_text)

In [ ]:
rcParams['figure.figsize'] = 10, 10
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

### Sentimental Analaysis -- Assigning Polarity to the Headlines

In [ ]:
# Create a function to get the subjectivity
def Subjectivity(text):
       return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def Polarity(text):
      return  TextBlob(text).sentiment.polarity

In [ ]:
news['Subjectivity'] =news['headline_text'].apply(Subjectivity)
news['Polarity'] =news['headline_text'].apply(Polarity)

In [ ]:
import nltk

senti = SentimentIntensityAnalyzer()

In [ ]:
news['Compound'] = [senti.polarity_scores(s)['compound'] for s in news['headline_text']]
news['Negative'] = [senti.polarity_scores(s)['neg'] for s in news['headline_text']]
news['Neutral'] = [senti.polarity_scores(s)['neu'] for s in news['headline_text']]
news['Positive'] = [senti.polarity_scores(s)['pos'] for s in news['headline_text']]


In [ ]:
news.head()

## Hybrid model - Combining Stocks data and news data

In [ ]:
news.rename(columns = {'publish_date':'Date'}, inplace = True)

In [ ]:
df_merge = pd.merge(stocks, news, how='inner', on=['Date'])
df_merge.head()

In [ ]:
df = df_merge[['Close','Subjectivity', 'Polarity', 'Compound', 'Negative', 'Neutral' ,'Positive']]
df.head()


In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
new_df = pd.DataFrame(sc.fit_transform(df))
new_df.columns = df.columns
new_df.index = df.index
new_df.head()

### Spliting Data into Train and test

In [ ]:
X = new_df.drop('Close', axis=1)
y =new_df['Close']


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state = 0)


In [ ]:
X_train.shape , X_test.shape

In [ ]:
def func_graph(results,names):
    fig = plt.figure()
    fig.suptitle('MSE value of all Algorithms Comparison')
    ax = fig.add_subplot(111)
    width = 0.5        
    bars=plt.bar(names,results, width, align='center')
    ax.set_xticklabels(names)
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x(), yval +0.005, yval)
    plt.show()

In [ ]:
from sklearn import metrics

def metric_calc(name,model,category, X_train, Y_train, X_test, Y_test):
    if category =='TRAINING DATA' :
        X_data= X_train
        Y_data=Y_train
    else :
        X_data= X_test
        Y_data=Y_test
        
    model.fit(X_train, Y_train)
    predictions = model.predict(X_data)
    mse =round(metrics.mean_squared_error(predictions,Y_data),4)   
    print('For ', name, 'MSE-Value is ', mse)
    return mse


In [ ]:
def func_modelling(i) :
    count=0
    count=count+1
    X = X_train[i]
    Y = Y_train
    x_test = X_test[i]
    seed = 7
    # preparing models list
    models = []
    models.append(('Decision Tree',' DecisiontreeRegressor  ', DecisionTreeRegressor()))
    models.append(('Random Forest',' RandomForestRegressor  ', RandomForestRegressor()))
    models.append(('XG Boost',' XGBRegressor  ', xgboost.XGBRegressor()))
    models.append(('LG Boost',' LGBMRegressor ', lightgbm.LGBMRegressor()))
    models.append(('ADA Boost',' AdaBoostRegressor ', AdaBoostRegressor()))
    results_train = []
    results_test = []
    names = []
    scoring = 'MSE'

    print('Metrics calcuated while TRANING the model')
    for name,label, model in models:
            cv_results_train=metric_calc(name,model,'TRAINING DATA',X,Y, x_test,Y_test)
            results_train.append(cv_results_train)
            names.append(name)
    func_graph(results_train,names)
    
    print('Evaluating the model on TESTING DATA')
    for name,label, model in models:
            cv_results_test=metric_calc(name,model,'TESTING DATA',X,Y, x_test,Y_test)
            results_test.append(cv_results_test)
            #names.append(name)
    func_graph(results_test,names)

### Training the model 

In [ ]:
func_modelling(X_train.columns)

LGBMRegressor has the least MSE and it has performed best for sentimental Anaylsis to predict if the stock close price with either increase or decrease depending on the news on that day.